In [4]:
# !pip install "qiskit[visualization,nature,algorithms]"
!pip install qiskit-nature qiskit-algorithms

In [5]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.converters import QubitConverter

from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Estimator as AerEstimator

# 1. 수소 분자 설정
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.735', basis='sto3g')
problem = ElectronicStructureProblem(driver)

# 2. Second-quantized Hamiltonian 생성
second_q_ops = problem.second_q_ops()
main_op = second_q_ops[0]

# 3. 매핑: Jordan-Wigner → Qubit 연산자
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper)
qubit_op = converter.convert(main_op, num_particles=problem.num_particles)

# 4. Ansatz 및 Optimizer
ansatz = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cx', reps=1)
optimizer = COBYLA(maxiter=100)
estimator = AerEstimator()  # Qiskit Aer 백엔드 사용

# 5. VQE 실행
vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

# 6. 결과 출력
print("H₂ 분자의 바닥상태 에너지 (VQE):", result.eigenvalue.real)


ModuleNotFoundError: No module named 'qiskit_nature.second_q.converters'

In [2]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.converters import QubitConverter

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit_nature.circuit.library import UCCSD
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Estimator
from qiskit_aer.primitives import Estimator as AerEstimator

# 1. 수소 분자 정의 (H-H 거리 0.735 Å)
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 .735', basis='sto3g')
es_problem = ElectronicStructureProblem(driver)

# 2. 해밀토니안 생성
second_q_op = es_problem.second_q_ops()
main_op = second_q_op[0]

# 3. 매핑 (페르미온 → 큐비트)
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper)
qubit_op = converter.convert(main_op, es_problem.num_particles)

# 4. Ansatz & Optimizer 정의
ansatz = TwoLocal(qubit_op.num_qubits, ['ry', 'rz'], 'cx', reps=1)
optimizer = COBYLA(maxiter=100)
estimator = AerEstimator()

# 5. VQE 알고리즘 구성 및 실행
vqe_solver = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
result = vqe_solver.compute_minimum_eigenvalue(qubit_op)

# 6. 결과 출력
print("H₂ 분자의 바닥상태 에너지 (VQE):", result.eigenvalue.real)


ModuleNotFoundError: No module named 'qiskit_nature.second_q.converters'